In [11]:
import __init__
#
from IPython.display import HTML, display
import pandas as pd
import numpy as np
import statsmodels.api as sm
#
from community_analysis import MIN_PICKUP_RATIO
#
def text_display(text, font_size):
    display(HTML('<font size=%d>' % font_size + text + '</font>'))   

In [2]:
def regression(dv, df):
    oc_dv = 'roamingTime' if dv == 'spendingTime' else 'spendingTime'
    rdf = df.copy(deep=True).drop([oc_dv], axis=1)
    candi_dummies = []
    num_iter = 1
    while True:
        for i, vs in enumerate(zip(*rdf.values)):
            if rdf.columns[i] == dv:
                continue
            if sum(vs) > len(rdf) * MIN_PICKUP_RATIO * num_iter:
                candi_dummies.append(rdf.columns[i])
        if len(rdf) <= len(candi_dummies):
            candi_dummies = []
            num_iter += 1
        else:
            break
    y = rdf[dv]
    X = rdf[candi_dummies]
    X = sm.add_constant(X)
    return sm.OLS(y, X, missing='drop').fit()

In [3]:
from community_analysis import tfZ_TP_dpath, tfZ_TP_prefix
#
df = pd.read_csv('%s/%s2009-3.csv' % (tfZ_TP_dpath, tfZ_TP_prefix))

In [5]:
df.head()

,month,day,timeFrame,zi,zj,tfZ,did,spendingTime,roamingTime,32768,...,32442,32469,32502,32509,32529,32537,32542,32647,32656,16375
0,1,1,14,54,12,"(14,54,12)",36942,240,240,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,14,38,45,"(14,38,45)",33720,48,238,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,14,61,27,"(14,61,27)",20677,592,1408,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,14,53,25,"(14,53,25)",29328,596,1428,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,14,41,16,"(14,41,16)",19360,356,987,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
text = '# of drivers in a file: %d' % len(set(df['did']))
text_display(text, 5)

In [6]:
did1 = 33720
did1_df = df[(df['did'] == did1)].copy(deep=True)
did1_df = did1_df.drop(['month', 'day', 'timeFrame', 'zi', 'zj', 'tfZ', 'did'], axis=1)
if '%d' % did1 in did1_df.columns:
    did1_df = did1_df.drop(['%d' % did1], axis=1)

In [8]:
text = '# of observations: %d' % len(did1_df)
text_display(text, 5)
text = '# of indepedent variables:: %d'% len(did1_df.columns)
text_display(text, 5)

In [13]:
text = 'Average: spendingTime %.2f, roamingTime %.2f' % (did1_df['spendingTime'].mean(), did1_df['roamingTime'].mean())
text_display(text, 5)

In [14]:
SP_res = regression('spendingTime', did1_df)
print SP_res.summary()

                            OLS Regression Results                            
Dep. Variable:           spendingTime   R-squared:                       0.654
Model:                            OLS   Adj. R-squared:                  0.392
Method:                 Least Squares   F-statistic:                     2.496
Date:                Thu, 19 Jan 2017   Prob (F-statistic):           1.62e-18
Time:                        08:58:50   Log-Likelihood:                -4534.3
No. Observations:                 734   AIC:                             9703.
Df Residuals:                     417   BIC:                         1.116e+04
Df Model:                         316                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         73.7327     12.441      5.927      0.0

In [15]:
RP_res = regression('roamingTime', did1_df)
print RP_res.summary()

                            OLS Regression Results                            
Dep. Variable:            roamingTime   R-squared:                       0.501
Model:                            OLS   Adj. R-squared:                  0.123
Method:                 Least Squares   F-statistic:                     1.324
Date:                Thu, 19 Jan 2017   Prob (F-statistic):            0.00372
Time:                        08:59:29   Log-Likelihood:                -5243.4
No. Observations:                 734   AIC:                         1.112e+04
Df Residuals:                     417   BIC:                         1.258e+04
Df Model:                         316                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        452.2424     32.692     13.834      0.0

In [23]:
header = ['D.V.', 'SP coef.', 'RP coef.', 'SP pv.', 'RP pv.']
df_data = {k: [] for k in header}
for dv, sp_pv in SP_res.pvalues.iteritems():
    for i, v in enumerate([dv, SP_res.params[dv], RP_res.params[dv], sp_pv, RP_res.pvalues[dv]]):
        df_data[header[i]].append(v)
df = pd.DataFrame(df_data)[header]
df

,D.V.,SP coef.,RP coef.,SP pv.,RP pv.
0,const,73.732734,452.242392,6.486451e-09,4.368426e-36
1,32880,32.720195,12.207599,4.206253e-01,9.089310e-01
2,32981,-11.218144,46.838510,7.744190e-01,6.488585e-01
3,33047,42.483696,193.691296,2.939620e-01,6.899525e-02
4,291,82.923667,93.139093,2.016401e-02,3.194005e-01
5,367,-64.306391,-8.653846,7.919562e-02,9.282374e-01
6,382,-21.514076,-53.692163,5.807321e-01,5.998830e-01
7,436,63.222936,82.582072,7.057973e-02,3.680442e-01
8,491,29.870710,87.569930,3.749469e-01,3.223001e-01
9,33298,85.421498,-19.015538,1.607998e-02,8.378666e-01
